In [ ]:
# ============================================================
# INSTALACIÓN DE LIBRERÍAS PARA COMPARACIÓN DEEPSPEED
# ============================================================

# 🔥 LIBRERÍAS ESENCIALES (obligatorias)
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install transformers  # Para modelos y tokenizers
%pip install datasets      # Para manejo de datasets
%pip install deepspeed    # ⭐ La estrella del show!
%pip install huggingface_hub   # Para manejo de datasets y modelos

# 🛠️ LIBRERÍAS DE SOPORTE (recomendadas)
%pip install accelerate    # Facilita integración con DeepSpeed
%pip install trl           # Para Supervised Fine-Tuning (SFT)

# 📊 MONITOREO Y VISUALIZACIÓN
%pip install psutil        # Monitor de CPU/RAM
%pip install GPUtil        # Monitor de GPU
%pip install matplotlib    # Gráficos
%pip install seaborn      # Gráficos bonitos
%pip install pandas       # Tablas de datos
%pip install numpy        # Operaciones numéricas

# 🔬 OPTIMIZACIONES AVANZADAS (opcionales para este ejemplo)
# %pip install peft           # Para LoRA/QLoRA (descomenta si quieres experimentar)
# %pip install bitsandbytes   # Para cuantización 8-bit/4-bit (descomenta si quieres experimentar)
# %pip install wandb          # Para logging avanzado (descomenta si tienes cuenta)


^C
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\impor\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [ ]:
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
from trl import SFTTrainer
import deepspeed
import time
import psutil
import GPUtil
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime
import os
import gc
import warnings
warnings.filterwarnings('ignore')

# Configurar para mostrar gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [ ]:
class PerformanceMonitor:
    """Clase para monitorear rendimiento durante el entrenamiento"""
    
    def __init__(self):
        self.metrics = {
            'gpu_memory_used': [],
            'gpu_utilization': [],
            'cpu_percent': [],
            'ram_used': [],
            'timestamps': []
        }
        
    def start_monitoring(self):
        """Inicia el monitoreo de recursos"""
        self.start_time = time.time()
        
    def log_metrics(self):
        """Registra métricas actuales del sistema"""
        # GPU metrics
        if torch.cuda.is_available():
            gpu = GPUtil.getGPUs()[0] if GPUtil.getGPUs() else None
            if gpu:
                self.metrics['gpu_memory_used'].append(gpu.memoryUsed)
                self.metrics['gpu_utilization'].append(gpu.load * 100)
            else:
                self.metrics['gpu_memory_used'].append(0)
                self.metrics['gpu_utilization'].append(0)
        else:
            self.metrics['gpu_memory_used'].append(0)
            self.metrics['gpu_utilization'].append(0)
            
        # CPU y RAM
        self.metrics['cpu_percent'].append(psutil.cpu_percent())
        self.metrics['ram_used'].append(psutil.virtual_memory().used / (1024**3))  # GB
        self.metrics['timestamps'].append(time.time() - self.start_time)
        
    def get_average_metrics(self):
        """Calcula métricas promedio"""
        return {
            'avg_gpu_memory': np.mean(self.metrics['gpu_memory_used']) if self.metrics['gpu_memory_used'] else 0,
            'avg_gpu_utilization': np.mean(self.metrics['gpu_utilization']) if self.metrics['gpu_utilization'] else 0,
            'avg_cpu_percent': np.mean(self.metrics['cpu_percent']) if self.metrics['cpu_percent'] else 0,
            'avg_ram_used': np.mean(self.metrics['ram_used']) if self.metrics['ram_used'] else 0,
            'total_time': self.metrics['timestamps'][-1] if self.metrics['timestamps'] else 0
        }

def print_system_info():
    """Muestra información del sistema"""
    print("🖥️ INFORMACIÓN DEL SISTEMA")
    print("=" * 50)
    print(f"🐍 Python: {torch.__version__}")
    print(f"🔥 PyTorch: {torch.__version__}")
    print(f"🚀 CUDA disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
        print(f"🎯 Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"💾 RAM total: {psutil.virtual_memory().total / (1024**3):.1f} GB")
    print(f"🔄 CPUs: {psutil.cpu_count()}")
    print("=" * 50)


In [ ]:
# ============================================================
# 📊 CREACIÓN DE DATASET SFT CON GEMINI 2.0 FLASH
# ============================================================

# Instalar Google Generative AI para Gemini 2.0 Flash
%pip install google-generativeai

import google.generativeai as genai
import random

# ⚠️ CONFIGURAR API KEY DE GEMINI
# Opción 1: Variable de entorno (recomendado)
import os
GEMINI_API_KEY = os.getenv('GOOGLE_API_KEY')

# Opción 2: Directamente (NO recomendado para producción)
if not GEMINI_API_KEY:
    print("⚠️  ATENCIÓN: Configura tu API key de Google Gemini")
    print("💡 Obtén tu key en: https://aistudio.google.com/app/apikey")
    GEMINI_API_KEY = input("🔑 Ingresa tu API key de Gemini: ").strip()

# Configurar Gemini
genai.configure(api_key=GEMINI_API_KEY)

def create_sft_dataset_with_gemini(num_samples=200):
    """
    Crea un dataset sintético de alta calidad usando Gemini 2.0 Flash
    para Supervised Fine-Tuning (SFT) de alineación de modelos.
    """
    
    print("🤖 Inicializando Gemini 2.0 Flash...")
    model = genai.GenerativeModel('gemini-2.0-flash-exp')
    
    # Categorías de datos para SFT de alineación
    sft_categories = [
        {
            "category": "helpful_assistant",
            "prompt": """Genera una conversación entre un usuario y un asistente IA útil, honesto y preciso. 
            El asistente debe dar respuestas detalladas, bien estructuradas y con ejemplos prácticos.
            Formato: Usuario: [pregunta] | Asistente: [respuesta completa y útil]"""
        },
        {
            "category": "safety_guidelines", 
            "prompt": """Genera una conversación donde el usuario hace una pregunta que requiere consideraciones de seguridad o ética,
            y el asistente responde de manera responsable, explicando los riesgos y alternativas seguras.
            Formato: Usuario: [pregunta con implicaciones de seguridad] | Asistente: [respuesta responsable y segura]"""
        },
        {
            "category": "educational_content",
            "prompt": """Genera una conversación educativa donde el usuario pregunta sobre conceptos técnicos complejos
            y el asistente explica de manera clara, progresiva y con analogías apropiadas.
            Formato: Usuario: [pregunta técnica] | Asistente: [explicación educativa clara]"""
        },
        {
            "category": "problem_solving",
            "prompt": """Genera una conversación donde el usuario presenta un problema específico
            y el asistente ofrece un enfoque estructurado de resolución paso a paso.
            Formato: Usuario: [problema específico] | Asistente: [solución estructurada paso a paso]"""
        },
        {
            "category": "refusal_harmful",
            "prompt": """Genera una conversación donde el usuario hace una solicitud inapropiada o dañina,
            y el asistente rechaza educadamente pero ofrece alternativas constructivas.
            Formato: Usuario: [solicitud inapropiada] | Asistente: [rechazo educado + alternativa constructiva]"""
        }
    ]
    
    dataset_entries = []
    samples_per_category = max(1, num_samples // len(sft_categories))
    
    print(f"🎯 Generando {num_samples} muestras de alta calidad...")
    print(f"📊 {samples_per_category} muestras por categoría")
    
    for category_info in sft_categories:
        category = category_info["category"]
        base_prompt = category_info["prompt"]
        
        print(f"\n🔄 Generando categoría: {category}")
        
        for i in range(samples_per_category):
            try:
                # Prompt específico para cada muestra
                full_prompt = f"""
                {base_prompt}
                
                INSTRUCCIONES ESPECÍFICAS:
                - Genera UNA conversación completa y realista
                - El usuario debe hacer una pregunta natural y específica
                - El asistente debe dar una respuesta completa, útil y bien estructurada
                - Usa un tono profesional pero accesible
                - La respuesta debe tener entre 100-300 palabras
                - Incluye ejemplos prácticos cuando sea apropiado
                
                IMPORTANTE: Responde SOLO con el formato solicitado, sin explicaciones adicionales.
                """
                
                # Generar contenido con Gemini
                response = model.generate_content(full_prompt)
                generated_text = response.text.strip()
                
                # Procesar la respuesta de Gemini
                if '|' in generated_text:
                    parts = generated_text.split('|', 1)
                    user_part = parts[0].replace('Usuario:', '').strip()
                    assistant_part = parts[1].replace('Asistente:', '').strip()
                    
                    # Formato estándar para SFT
                    conversation = f"<|user|>\n{user_part}\n<|assistant|>\n{assistant_part}"
                    
                    dataset_entries.append({
                        "text": conversation,
                        "category": category,
                        "length": len(conversation),
                        "tokens_estimate": len(conversation.split())
                    })
                    
                    print(f"  ✅ Muestra {i+1}/{samples_per_category} generada")
                else:
                    print(f"  ⚠️  Formato incorrecto en muestra {i+1}, reintentando...")
                    
            except Exception as e:
                print(f"  ❌ Error generando muestra {i+1}: {str(e)}")
                continue
    
    # Crear dataset de Hugging Face
    print(f"\n📦 Creando dataset con {len(dataset_entries)} muestras válidas...")
    dataset = Dataset.from_list(dataset_entries)
    
    return dataset

def show_dataset_stats(dataset):
    """Muestra estadísticas detalladas del dataset"""
    print("\n📈 ESTADÍSTICAS DEL DATASET SFT")
    print("=" * 60)
    
    # Estadísticas básicas
    print(f"📊 Total de muestras: {len(dataset)}")
    print(f"📏 Longitud promedio: {np.mean([item['length'] for item in dataset]):.1f} caracteres")
    print(f"🎯 Tokens estimados promedio: {np.mean([item['tokens_estimate'] for item in dataset]):.1f}")
    
    # Distribución por categorías
    if 'category' in dataset.column_names:
        categories = {}
        for item in dataset:
            cat = item['category']
            categories[cat] = categories.get(cat, 0) + 1
        
        print("\n🏷️  DISTRIBUCIÓN POR CATEGORÍAS:")
        for cat, count in categories.items():
            print(f"  • {cat}: {count} muestras")
    
    print("\n📝 EJEMPLO DE CONVERSACIÓN GENERADA:")
    print("-" * 60)
    example = dataset[0]['text']
    print(example[:400] + "..." if len(example) > 400 else example)
    print("-" * 60)

# 🚀 GENERAR DATASET CON GEMINI 2.0 FLASH
print("🚀 CREANDO DATASET SFT CON GEMINI 2.0 FLASH")
print("=" * 60)

try:
    dataset = create_sft_dataset_with_gemini(num_samples=100)  # Empezamos con 100 para pruebas
    show_dataset_stats(dataset)
    
    print("\n✅ ¡Dataset SFT de alta calidad creado exitosamente!")
    print("🎯 Perfecto para entrenar modelos alineados y útiles")
    
except Exception as e:
    print(f"❌ Error creando dataset: {str(e)}")
    print("\n🔄 Creando dataset de respaldo básico...")
    
    # Dataset de respaldo si Gemini falla
    fallback_data = [
        {
            "text": "<|user|>\n¿Cómo puedo implementar una función recursiva en Python?\n<|assistant|>\nUna función recursiva es aquella que se llama a sí misma. Aquí te explico cómo implementarla:\n\n1. **Caso base**: Condición que detiene la recursión\n2. **Caso recursivo**: La función se llama a sí misma\n\nEjemplo - Factorial:\n```python\ndef factorial(n):\n    if n <= 1:  # Caso base\n        return 1\n    return n * factorial(n-1)  # Caso recursivo\n```\n\n**Consejos importantes**:\n- Siempre define un caso base claro\n- Asegúrate de que la recursión progrese hacia el caso base\n- Considera el límite de recursión de Python (1000 por defecto)", 
            "category": "educational", 
            "length": 500, 
            "tokens_estimate": 120
        }
    ]
    dataset = Dataset.from_list(fallback_data)
    show_dataset_stats(dataset)

In [ ]:
# Información del sistema
print_system_info()

# Configuración del modelo
MODEL_NAME = "microsoft/DialoGPT-small"  # Modelo pequeño para demostración
MAX_LENGTH = 256
BATCH_SIZE = 4
NUM_EPOCHS = 1
LEARNING_RATE = 2e-5

print(f"\\n🤖 Configuración del modelo:")
print(f"📦 Modelo: {MODEL_NAME}")
print(f"📏 Longitud máxima: {MAX_LENGTH}")
print(f"🔢 Batch size: {BATCH_SIZE}")
print(f"🔄 Épocas: {NUM_EPOCHS}")
print(f"📈 Learning rate: {LEARNING_RATE}")

# Cargar tokenizer
print("\\n🔤 Cargando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ Tokenizer cargado correctamente")


In [ ]:
def preprocess_function(examples):
    """Preprocesa los datos para el entrenamiento"""
    # Tokenizar los textos
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        return_tensors="pt"
    )
    
    # Para language modeling, labels = input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

# Preprocesar dataset
print("🔄 Preprocesando dataset...")
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

# Dividir en train/test
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"✅ Datos preparados:")
print(f"🎯 Training samples: {len(train_dataset)}")
print(f"📊 Evaluation samples: {len(eval_dataset)}")

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling para GPT
)


## ⚡ Paso 8: Configuración de DeepSpeed

In [ ]:
# Crear configuración de DeepSpeed
deepspeed_config = {
    "train_batch_size": BATCH_SIZE,
    "train_micro_batch_size_per_gpu": BATCH_SIZE,
    "gradient_accumulation_steps": 1,
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": LEARNING_RATE,
            "betas": [0.9, 0.999],
            "eps": 1e-8,
            "weight_decay": 0.01
        }
    },
    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": 0,
            "warmup_max_lr": LEARNING_RATE,
            "warmup_num_steps": 50
        }
    },
    "fp16": {
        "enabled": True,
        "loss_scale": 0,
        "initial_scale_power": 16,
        "loss_scale_window": 1000,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "zero_optimization": {
        "stage": 2,
        "allgather_partitions": True,
        "allgather_bucket_size": 2e8,
        "reduce_scatter": True,
        "reduce_bucket_size": 2e8,
        "overlap_comm": True,
        "contiguous_gradients": True
    },
    "gradient_clipping": 1.0,
    "wall_clock_breakdown": False
}

# Guardar configuración
with open("deepspeed_config.json", "w") as f:
    json.dump(deepspeed_config, f, indent=2)

print("⚡ CONFIGURACIÓN DEEPSPEED CREADA")
print("=" * 40)
print("🎯 Optimizaciones habilitadas:")
print("  ✅ FP16 (Half Precision)")
print("  ✅ ZeRO Stage 2 (Optimizer State Partitioning)")
print("  ✅ Gradient Clipping")
print("  ✅ Comunicación Optimizada")
print("  ✅ AdamW Optimizer")
print("  ✅ Warmup Learning Rate Scheduler")
print("=" * 40)


In [ ]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, TaskType

def train_baseline_model():
    """Entrena el modelo SIN DeepSpeed usando SFTTrainer + LoRA"""
    
    print("🐌 INICIANDO ENTRENAMIENTO BASELINE (SIN DEEPSPEED)")
    print("=" * 60)
    
    # Limpiar memoria GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    # Configuración LoRA para PEFT
    peft_config = LoraConfig(
        r=16,                           # Rango de LoRA
        lora_alpha=32,                  # Parámetro de escalado
        lora_dropout=0.05,              # Dropout para regularización
        target_modules="all-linear",    # Aplicar a todas las capas lineales
        modules_to_save=["lm_head", "embed_tokens"],  # Módulos a guardar completos
        task_type=TaskType.CAUSAL_LM,   # Tipo de tarea
        bias="none"                     # No entrenar bias
    )
    
    # Configuración de entrenamiento SFT
    sft_config = SFTConfig(
        output_dir="./baseline_sft_results",
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        warmup_steps=50,
        logging_steps=20,
        eval_strategy="steps",
        eval_steps=50,
        save_steps=100,
        save_total_limit=2,
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        report_to=None,
        # Configuraciones específicas de SFT
        max_seq_length=MAX_LENGTH,
        packing=False,                  # No empaquetar secuencias para simplicidad
        dataset_text_field="text",      # Campo que contiene el texto
        seed=42
    )
    
    print("🤖 Cargando modelo base...")
    print(f"📦 Modelo: {MODEL_NAME}")
    
    # Inicializar monitor de rendimiento
    monitor = PerformanceMonitor()
    monitor.start_monitoring()
    
    # Crear SFTTrainer (carga el modelo automáticamente)
    trainer = SFTTrainer(
        model=MODEL_NAME,              # Cargar directamente por nombre
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        args=sft_config,
        peft_config=peft_config,       # Configuración LoRA
        tokenizer=tokenizer,
        data_collator=None,            # SFTTrainer maneja esto automáticamente
    )
    
    print("✅ SFTTrainer configurado con LoRA")
    print(f"🎯 Parámetros entrenables: {trainer.model.print_trainable_parameters()}")
    
    # Entrenar
    print("🚀 Iniciando entrenamiento SFT...")
    start_time = time.time()
    
    # Callback para monitoreo durante entrenamiento
    class MonitoringCallback:
        def __init__(self, monitor):
            self.monitor = monitor
            
        def on_step_end(self, args, state, control, logs=None, **kwargs):
            self.monitor.log_metrics()
    
    trainer.add_callback(MonitoringCallback(monitor))
    
    # ¡Entrenar!
    trainer.train()
    
    end_time = time.time()
    training_time = end_time - start_time
    
    # Obtener métricas finales
    final_metrics = monitor.get_average_metrics()
    final_metrics['total_training_time'] = training_time
    
    print(f"\n✅ Entrenamiento baseline completado en {training_time:.2f} segundos")
    
    # Guardar el modelo entrenado
    trainer.save_model("./baseline_sft_final")
    print("💾 Modelo baseline guardado en ./baseline_sft_final")
    
    # Limpiar memoria
    del trainer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    return final_metrics, monitor.metrics

# ============================================================
# 🚀 PASO 9: ENTRENAMIENTO CON DEEPSPEED 
# ============================================================

def train_deepspeed_model():
    """Entrena el modelo CON DeepSpeed usando SFTTrainer + LoRA"""
    
    print("🚀 INICIANDO ENTRENAMIENTO CON DEEPSPEED")
    print("=" * 60)
    
    # Limpiar memoria GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    # Misma configuración LoRA (para comparación justa)
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules="all-linear",
        modules_to_save=["lm_head", "embed_tokens"],
        task_type=TaskType.CAUSAL_LM,
        bias="none"
    )
    
    # Configuración SFT con DeepSpeed
    sft_config = SFTConfig(
        output_dir="./deepspeed_sft_results",
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        warmup_steps=50,
        logging_steps=20,
        eval_strategy="steps",
        eval_steps=50,
        save_steps=100,
        save_total_limit=2,
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        report_to=None,
        # Configuraciones específicas de SFT
        max_seq_length=MAX_LENGTH,
        packing=False,
        dataset_text_field="text",
        seed=42,
        # ⚡ CONFIGURACIÓN DEEPSPEED
        deepspeed="deepspeed_config.json",  # ¡Aquí está la magia!
        fp16=True,                          # Half precision
        dataloader_num_workers=0,           # Para evitar problemas con DeepSpeed
    )
    
    print("🤖 Cargando modelo base con optimizaciones DeepSpeed...")
    print(f"📦 Modelo: {MODEL_NAME}")
    print("⚡ Optimizaciones: ZeRO Stage 2 + FP16 + LoRA")
    
    # Inicializar monitor de rendimiento
    monitor = PerformanceMonitor()
    monitor.start_monitoring()
    
    # Crear SFTTrainer con DeepSpeed
    trainer = SFTTrainer(
        model=MODEL_NAME,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        args=sft_config,
        peft_config=peft_config,
        tokenizer=tokenizer,
        data_collator=None,
    )
    
    print("✅ SFTTrainer configurado con DeepSpeed + LoRA")
    print(f"🎯 Parámetros entrenables: {trainer.model.print_trainable_parameters()}")
    
    # Entrenar
    print("🚀 Iniciando entrenamiento SFT con DeepSpeed...")
    start_time = time.time()
    
    # Callback para monitoreo
    class MonitoringCallback:
        def __init__(self, monitor):
            self.monitor = monitor
            
        def on_step_end(self, args, state, control, logs=None, **kwargs):
            self.monitor.log_metrics()
    
    trainer.add_callback(MonitoringCallback(monitor))
    
    # ¡Entrenar con DeepSpeed!
    trainer.train()
    
    end_time = time.time()
    training_time = end_time - start_time
    
    # Obtener métricas finales
    final_metrics = monitor.get_average_metrics()
    final_metrics['total_training_time'] = training_time
    
    print(f"\n✅ Entrenamiento con DeepSpeed completado en {training_time:.2f} segundos")
    
    # Guardar el modelo entrenado
    trainer.save_model("./deepspeed_sft_final")
    print("💾 Modelo DeepSpeed guardado en ./deepspeed_sft_final")
    
    # Limpiar memoria
    del trainer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    return final_metrics, monitor.metrics

# ============================================================
# 🔄 EJECUTAR ENTRENAMIENTOS
# ============================================================

print("🎯 CONFIGURACIÓN DEL EXPERIMENTO:")
print(f"📦 Modelo base: {MODEL_NAME}")
print(f"📊 Dataset size: {len(train_dataset)} samples")
print(f"🔢 Batch size: {BATCH_SIZE}")
print(f"🔄 Épocas: {NUM_EPOCHS}")
print(f"📏 Max length: {MAX_LENGTH}")
print(f"🎯 LoRA rank: 16")

# Ejecutar entrenamiento baseline
print("\n" + "="*80)
baseline_metrics, baseline_detailed = train_baseline_model()

print("\n📊 MÉTRICAS BASELINE:")
print(f"⏱️  Tiempo total: {baseline_metrics['total_training_time']:.2f}s")
print(f"🎯 GPU memoria promedio: {baseline_metrics['avg_gpu_memory']:.1f} MB")
print(f"📈 GPU utilización promedio: {baseline_metrics['avg_gpu_utilization']:.1f}%")
print(f"🔄 CPU promedio: {baseline_metrics['avg_cpu_percent']:.1f}%")
print(f"💾 RAM promedio: {baseline_metrics['avg_ram_used']:.1f} GB")

# Ejecutar entrenamiento con DeepSpeed
print("\n" + "="*80)
deepspeed_metrics, deepspeed_detailed = train_deepspeed_model()

print("\n📊 MÉTRICAS DEEPSPEED:")
print(f"⏱️  Tiempo total: {deepspeed_metrics['total_training_time']:.2f}s")
print(f"🎯 GPU memoria promedio: {deepspeed_metrics['avg_gpu_memory']:.1f} MB")
print(f"📈 GPU utilización promedio: {deepspeed_metrics['avg_gpu_utilization']:.1f}%")
print(f"🔄 CPU promedio: {deepspeed_metrics['avg_cpu_percent']:.1f}%")
print(f"💾 RAM promedio: {deepspeed_metrics['avg_ram_used']:.1f} GB")


In [ ]:
def calculate_improvements(baseline, deepspeed):
    """Calcula las mejoras de DeepSpeed vs baseline"""
    improvements = {}
    
    # Tiempo de entrenamiento (menor es mejor)
    time_improvement = ((baseline['total_training_time'] - deepspeed['total_training_time']) / 
                       baseline['total_training_time']) * 100
    improvements['time_speedup'] = time_improvement
    
    # Memoria GPU (menor es mejor)
    memory_improvement = ((baseline['avg_gpu_memory'] - deepspeed['avg_gpu_memory']) / 
                         baseline['avg_gpu_memory']) * 100
    improvements['memory_reduction'] = memory_improvement
    
    # Utilización GPU (mayor es mejor, pero calculamos eficiencia)
    gpu_efficiency = (deepspeed['avg_gpu_utilization'] / baseline['avg_gpu_utilization'] - 1) * 100
    improvements['gpu_efficiency'] = gpu_efficiency
    
    return improvements

# Calcular mejoras
improvements = calculate_improvements(baseline_metrics, deepspeed_metrics)

# Crear tabla de comparación
comparison_data = {
    'Métrica': [
        'Tiempo de Entrenamiento (s)',
        'Memoria GPU Promedio (MB)',
        'Utilización GPU Promedio (%)',
        'CPU Promedio (%)',
        'RAM Promedio (GB)'
    ],
    'Baseline (Sin DeepSpeed)': [
        f"{baseline_metrics['total_training_time']:.2f}",
        f"{baseline_metrics['avg_gpu_memory']:.1f}",
        f"{baseline_metrics['avg_gpu_utilization']:.1f}",
        f"{baseline_metrics['avg_cpu_percent']:.1f}",
        f"{baseline_metrics['avg_ram_used']:.1f}"
    ],
    'DeepSpeed Optimizado': [
        f"{deepspeed_metrics['total_training_time']:.2f}",
        f"{deepspeed_metrics['avg_gpu_memory']:.1f}",
        f"{deepspeed_metrics['avg_gpu_utilization']:.1f}",
        f"{deepspeed_metrics['avg_cpu_percent']:.1f}",
        f"{deepspeed_metrics['avg_ram_used']:.1f}"
    ],
    'Mejora (%)': [
        f"{improvements['time_speedup']:+.1f}",
        f"{improvements['memory_reduction']:+.1f}",
        f"{improvements['gpu_efficiency']:+.1f}",
        "N/A",
        "N/A"
    ]
}

comparison_df = pd.DataFrame(comparison_data)

print("🏆 COMPARACIÓN DE RESULTADOS")
print("=" * 80)
print(comparison_df.to_string(index=False))
print("=" * 80)

# Resumen de beneficios
print("\\n🎯 RESUMEN DE BENEFICIOS DE DEEPSPEED:")
print(f"⚡ Aceleración del entrenamiento: {improvements['time_speedup']:+.1f}%")
print(f"💾 Reducción de memoria GPU: {improvements['memory_reduction']:+.1f}%")
print(f"📈 Mejora en eficiencia GPU: {improvements['gpu_efficiency']:+.1f}%")

if improvements['time_speedup'] > 0:
    print(f"\\n🚀 El entrenamiento fue {improvements['time_speedup']:.1f}% más rápido con DeepSpeed!")
if improvements['memory_reduction'] > 0:
    print(f"💡 Se redujo el uso de memoria GPU en {improvements['memory_reduction']:.1f}%")


In [ ]:
# Crear visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🚀 Comparación de Rendimiento: Baseline vs DeepSpeed', fontsize=16, fontweight='bold')

# 1. Tiempo de entrenamiento
ax1 = axes[0, 0]
methods = ['Baseline\\n(Sin DeepSpeed)', 'DeepSpeed\\n(Optimizado)']
times = [baseline_metrics['total_training_time'], deepspeed_metrics['total_training_time']]
colors = ['#ff7f7f', '#7fbf7f']

bars1 = ax1.bar(methods, times, color=colors, alpha=0.8, edgecolor='black', linewidth=1)
ax1.set_title('⏱️ Tiempo de Entrenamiento', fontweight='bold')
ax1.set_ylabel('Tiempo (segundos)')
ax1.grid(True, alpha=0.3)

# Agregar valores en las barras
for bar, time in zip(bars1, times):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{time:.1f}s', ha='center', va='bottom', fontweight='bold')

# 2. Memoria GPU
ax2 = axes[0, 1]
memory_usage = [baseline_metrics['avg_gpu_memory'], deepspeed_metrics['avg_gpu_memory']]

bars2 = ax2.bar(methods, memory_usage, color=colors, alpha=0.8, edgecolor='black', linewidth=1)
ax2.set_title('🎯 Uso de Memoria GPU', fontweight='bold')
ax2.set_ylabel('Memoria (MB)')
ax2.grid(True, alpha=0.3)

for bar, memory in zip(bars2, memory_usage):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{memory:.0f}MB', ha='center', va='bottom', fontweight='bold')

# 3. Utilización GPU
ax3 = axes[1, 0]
gpu_util = [baseline_metrics['avg_gpu_utilization'], deepspeed_metrics['avg_gpu_utilization']]

bars3 = ax3.bar(methods, gpu_util, color=colors, alpha=0.8, edgecolor='black', linewidth=1)
ax3.set_title('📈 Utilización GPU', fontweight='bold')
ax3.set_ylabel('Utilización (%)')
ax3.set_ylim(0, 100)
ax3.grid(True, alpha=0.3)

for bar, util in zip(bars3, gpu_util):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 2,
             f'{util:.1f}%', ha='center', va='bottom', fontweight='bold')

# 4. Gráfico de mejoras
ax4 = axes[1, 1]
improvement_metrics = ['Velocidad\\nEntrenamiento', 'Reducción\\nMemoria GPU']
improvement_values = [improvements['time_speedup'], improvements['memory_reduction']]
improvement_colors = ['green' if x > 0 else 'red' for x in improvement_values]

bars4 = ax4.bar(improvement_metrics, improvement_values, color=improvement_colors, alpha=0.7, edgecolor='black', linewidth=1)
ax4.set_title('🏆 Mejoras con DeepSpeed', fontweight='bold')
ax4.set_ylabel('Mejora (%)')
ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax4.grid(True, alpha=0.3)

for bar, improvement in zip(bars4, improvement_values):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + (1 if height > 0 else -3),
             f'{improvement:+.1f}%', ha='center', va='bottom' if height > 0 else 'top', fontweight='bold')

plt.tight_layout()
plt.show()

# Gráfico de línea temporal para memoria GPU
if len(baseline_detailed['timestamps']) > 0 and len(deepspeed_detailed['timestamps']) > 0:
    plt.figure(figsize=(12, 6))
    
    plt.plot(baseline_detailed['timestamps'], baseline_detailed['gpu_memory_used'], 
             label='Baseline (Sin DeepSpeed)', color='red', linewidth=2, alpha=0.8)
    plt.plot(deepspeed_detailed['timestamps'], deepspeed_detailed['gpu_memory_used'], 
             label='DeepSpeed (Optimizado)', color='green', linewidth=2, alpha=0.8)
    
    plt.title('📈 Uso de Memoria GPU Durante el Entrenamiento', fontsize=14, fontweight='bold')
    plt.xlabel('Tiempo (segundos)')
    plt.ylabel('Memoria GPU (MB)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
